# DATA EDA

### Imports

In [1]:
#Imports
import pandas as pd

### Dataframes Imports 

In [2]:
#Importing Transactions data
transactions_df= pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/transactions_v2.csv")
#Importing user logs data
user_logs_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/user_logs_v2.csv")
#importing members data
members_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/members_v3.csv")
# import training dataset
train_df = pd.read_csv("/Users/andretomaz/code/XavierLooyens/Churn_prediction/raw_data/train_v2.csv")

### Transactions Data

In [4]:
# converting transaction_date and membership_date to datetime object
transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'], format='%Y%m%d')
transactions_df['membership_expire_date'] = pd.to_datetime(transactions_df['membership_expire_date'], format='%Y%m%d')

In [5]:
# removing duplicates and leaving only the latest transaction date
transactions_df_lt = transactions_df.loc[transactions_df.groupby('msno').transaction_date.idxmax()]

In [6]:
# creating remaining plan duration column
transactions_df_lt['remaining_plan_duration'] = transactions_df_lt['membership_expire_date'] - transactions_df_lt['transaction_date']

In [7]:
# creating discount column
transactions_df_lt['is_discount'] = transactions_df_lt.apply(lambda x: '0' if (x['actual_amount_paid'] -x['plan_list_price'])>=0 else '1', axis=1)

### User logs Data

In [10]:
# Converting date into datetime object
user_logs_df['date'] = pd.to_datetime(user_logs_df['date'], format='%Y%m%d')

In [18]:
#create new dataframe from transactions table with only msno and latest transaction date
latest_transactions_per_msno = transactions_df_lt[['msno', 'transaction_date']]

In [21]:
#Merge user logs with latest transaction date
merged_df = user_logs_df.merge(latest_transactions_per_msno, on='msno', how='left')

In [27]:
# drop msno's where transaction date is NaN
merged_df = merged_df.dropna(subset=['transaction_date'])

In [41]:
#removing rows where the user log data is before the last transaction date
user_logs_atd= merged_df.loc[merged_df['date']>=merged_df['transaction_date']]

In [42]:
# removing data column
user_logs_atd = user_logs_atd.drop(columns=['date','transaction_date'])

In [43]:
#groupby msno and summing all values
user_logs_atd = user_logs_atd.groupby('msno').sum()

### Merging

In [51]:
# Merging transactions, user logs and members data to Train dataframe
train_df = train_df.merge(transactions_df_lt, on='msno', how='left')
train_df = train_df.merge(user_logs_atd, on='msno', how='left')
train_df = train_df.merge(members_df, on='msno', how='left')

In [52]:
train_df

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel,...,num_75,num_985,num_100,num_unq,total_secs,city,bd,gender,registered_via,registration_init_time
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0,28.0,male,3.0,20131223.0
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36.0,30.0,180.0,180.0,0.0,2017-03-11,2017-04-11,0.0,...,1.0,0.0,14.0,16.0,3648.832,13.0,20.0,male,3.0,20131223.0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15.0,90.0,300.0,300.0,0.0,2017-03-14,2017-06-15,0.0,...,17.0,16.0,136.0,237.0,43621.799,13.0,18.0,male,3.0,20131227.0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41.0,30.0,149.0,149.0,1.0,2015-12-08,2017-12-08,0.0,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,7.0,20140109.0
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1,41.0,30.0,99.0,99.0,1.0,2017-03-16,2017-09-18,0.0,...,3.0,3.0,738.0,395.0,186215.053,13.0,35.0,female,7.0,20140125.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970955,OHnZbu+EVaP+vN7Z+OfT5OMcp90MWFZonmM0o3pb8FY=,0,40.0,30.0,149.0,149.0,1.0,2017-03-04,2017-04-03,0.0,...,4.0,8.0,276.0,258.0,67346.703,13.0,28.0,male,7.0,20110831.0
970956,S92bDK//uI6hk3u1vuApro0qJBQOToBozZ7lL1yTC+w=,0,41.0,30.0,99.0,99.0,1.0,2017-03-10,2017-04-10,0.0,...,5.0,4.0,14.0,19.0,5596.102,1.0,0.0,NaN,7.0,20110903.0
970957,eUa3xo16vpAjr43Cjlb6Kjf1NTILYyJIkBayJQdXWnw=,0,41.0,30.0,99.0,99.0,1.0,2017-03-16,2017-04-16,0.0,...,NaN,NaN,NaN,NaN,NaN,1.0,0.0,NaN,7.0,20110905.0
970958,iZE41tbAQ65rJq60olkJT4BJzuUAYgQdfbEemXe/TTk=,0,39.0,30.0,149.0,149.0,1.0,2017-03-31,2017-05-13,0.0,...,NaN,NaN,NaN,NaN,NaN,5.0,25.0,female,9.0,20110905.0


### Exploration

In [53]:
train_df.isnull().sum()

msno                            0
is_churn                        0
payment_method_id           37382
payment_plan_days           37382
plan_list_price             37382
actual_amount_paid          37382
is_auto_renew               37382
transaction_date            37382
membership_expire_date      37382
is_cancel                   37382
remaining_plan_duration     37382
is_discount                 37382
num_25                     334219
num_50                     334219
num_75                     334219
num_985                    334219
num_100                    334219
num_unq                    334219
total_secs                 334219
city                       109993
bd                         109993
gender                     582055
registered_via             109993
registration_init_time     109993
dtype: int64